# Best Submission Builder
This notebook allows to load a different model for each team and plant in order to make a more precise prediction on each of the datasets. Hence, it is important to have the checkpoint of each model, load them and prepare the filenames for the predict. In the end the submission dictionary is written in the json used for the submission

## Notebook settings

In [ ]:
import os
import tensorflow as tf
import numpy as np
import json
from PIL import Image

SEED = 1234
tf.random.set_seed(SEED)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cwd = os.getcwd() # should be /content
dataset_version = 'Final_Filtered_Dataset_512'
dataset_version_noTile = 'Final_Dataset'

if not os.path.exists(os.path.join(cwd, dataset_version)):
  !unzip '/content/drive/My Drive/challenge2/dataset/Final_Filtered_Dataset_512.zip'

if not os.path.exists(os.path.join(cwd, dataset_version_noTile)):
  !unzip '/content/drive/My Drive/challenge2/dataset/Final_Dataset.zip'

## Prepare for loading
In order to load back the models we need to specify all the custom object used at compile time and pass them to the __load_model__ method provided by Keras.

### Intersection Over Union

In [ ]:
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,num_classes): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [ ]:
def cropIoU(y_true, y_pred):
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)
    i = 1
    class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
    class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
    intersection = tf.reduce_sum(class_true * class_pred)
    union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    iou = (intersection + 1e-7) / (union + 1e-7)
    return iou

def weedIoU(y_true, y_pred):
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)
    i = 2
    class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
    class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
    intersection = tf.reduce_sum(class_true * class_pred)
    union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    iou = (intersection + 1e-7) / (union + 1e-7)
    return iou

### Weighted Categorical Crossentropy

In [ ]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)



    def loss(y_true, y_pred):
      #print("y_true shape={}\ny_pred shape={}\n{}".format(y_true.shape, y_pred.shape, y_true[:,:,:]))  
      # scale predictions so that the class probas of each sample sum to 1
      y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
      # clip to prevent NaN's and Inf's
      y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
      #trasformo da sparse representation a onehot representation per calcolare la loss
      #one_hot_encoding = np.zeros((y_true.shape[0], y_true.shape[1], y_true.shape[2], y_pred.shape[3]))
      #one_hot_encoding = K.zeros((y_true.shape[0], y_true.shape[1], y_true.shape[2], y_pred.shape[3]))
      one_hot_encoding = (tf.cast(tf.reduce_any(y_true == 0, axis=-1, keepdims=True), tf.float32)*tf.constant([1.,0.,0.]) +
                          tf.cast(tf.reduce_any(y_true == 1, axis=-1, keepdims=True), tf.float32)*tf.constant([0.,1.,0.])+
                          tf.cast(tf.reduce_any(y_true == 2, axis=-1, keepdims=True), tf.float32)*tf.constant([0.,0.,1.]))
                    

      # calc
      #se  non fosse one hot avrei tipo [[0,1,0],[0,0,1]] * [[a,b,c], [d,e,f]] = [[a,b,c], [2d,2e,2f]], ma io voglio [[0,b,0], [0,0,f]]
      #1 2 
      loss = one_hot_encoding * K.log(y_pred) * weights
      loss = -K.sum(loss, -1)
      return loss
    
    return loss

## Prepare for Testing
Since different models should be loaed we have to prepare all the methods needed to make the predict. Hence both the scripts for testing a model on the tiles and the ones for testing on the full images.

In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
# tiling utils
tile_size = 512

def get_patches(img_arr, size=256, stride=256):

    patches_list = []
    i_max = img_arr.shape[0] // stride
    j_max = img_arr.shape[1] // stride

    for i in range(i_max):
        for j in range(j_max):
            patches_list.append(
                img_arr[
                    i * stride : i * stride + size,
                    j * stride : j * stride + size
                ]
            )

    return np.stack(patches_list)

def reconstruct_from_patches(img_arr, org_img_size, stride, size):

    if img_arr.ndim == 3:
        img_arr = np.expand_dims(img_arr, axis=0)

    if size is None:
        size = img_arr.shape[1]

    if stride is None:
        stride = size

    nm_layers = img_arr.shape[3]

    i_max = (org_img_size[0] // stride) + 1 - (size // stride)
    j_max = (org_img_size[1] // stride) + 1 - (size // stride)

    total_nm_images = img_arr.shape[0] // (j_max * i_max)
    nm_images = img_arr.shape[0]

    averaging_value = size // stride
    images_list = []
    kk = 0
    for img_count in range(total_nm_images):
        img_bg = np.zeros(
            (org_img_size[0], org_img_size[1], nm_layers), dtype=img_arr[0].dtype
        )

        for i in range(i_max):
            for j in range(j_max):
                for layer in range(nm_layers):
                    img_bg[
                        i * stride : i * stride + size,
                        j * stride : j * stride + size,
                        layer,
                    ] = img_arr[kk, :, :, layer]

                kk += 1

        images_list.append(img_bg)
    return np.stack(images_list)

### Load the Models
The following cell allows to load the models once we provide the path containing the __saved_model.pb__ file of the respective model. Remember that for the specific model you are loading you need to specify in the __custom_objects__ dictionary all the custom methods that were used at compile time. To split the prediction of each model on the respective dataset we build two lists in this order: __bh - bm - ph - pm - rh - rm - wh - wm__. The lists contain:
* __models_list__: list of all the loaded models
* __models_images_filenames__: list of lists containing the filenames of all the images on which the corresponding model will predict

In [ ]:
# Order is: bh - bm - ph - pm - rh - rm - wh - wm
models_list = []
models_images_filenames = []

# you can generate these checkpoints by running the specified notebooks in the proper way, or you can download them from this link:
# https://drive.google.com/drive/folders/1jvHfUNKOFcReSQLgm0IA3M79-PmXSSNc?usp=sharing

path_model1 = 'checkpoint generated from unet_ADD_SKIP notebook, including only Bipbip Haricot'
path_model2 = 'checkpoint generated from UNET_VGG notebook, excluding Pead team'
path_model3 = 'checkpoint generated from unet_ADD_SKIP notebook, including only Pead Haricot'
path_model4 = 'checkpoint generated from UNET_PEAD_MAIS notebook, inlcuding only Pead Mais'
path_model5 = 'checkpoint generated from unet_ADD_SKIP notebook, including only Weedelec Haricot'

# bipbip models
bh_model = tf.keras.models.load_model(path_model1, custom_objects={"meanIoU": meanIoU, "loss":weighted_categorical_crossentropy(np.asarray([0.5,1.,1.5], dtype=np.float32))}) 
bm_model = tf.keras.models.load_model(path_model2, custom_objects={"meanIoU": meanIoU, "cropIoU": cropIoU, "loss":weighted_categorical_crossentropy(np.asarray([0.5,1.,2], dtype=np.float32))})
models_list.append(bh_model)
models_list.append(bm_model)

# pead models
ph_model = tf.keras.models.load_model(path_model3, custom_objects={"meanIoU": meanIoU, "loss":weighted_categorical_crossentropy(np.asarray([0.5,1.,1.5], dtype=np.float32))})
pm_model = tf.keras.models.load_model(path_model4, custom_objects={"meanIoU": meanIoU, "cropIoU": cropIoU, "weedIoU": weedIoU})
models_list.append(ph_model)
models_list.append(pm_model)

# roseau models
rh_model = tf.keras.models.load_model(path_model2, custom_objects={"meanIoU": meanIoU, "cropIoU": cropIoU, "loss":weighted_categorical_crossentropy(np.asarray([0.5,1.,2], dtype=np.float32))})
rm_model = tf.keras.models.load_model(path_model2, custom_objects={"meanIoU": meanIoU, "cropIoU": cropIoU, "loss":weighted_categorical_crossentropy(np.asarray([0.5,1.,2], dtype=np.float32))})
models_list.append(rh_model)
models_list.append(rm_model)

# weedelec models
wh_model = tf.keras.models.load_model(path_model5, custom_objects={"meanIoU": meanIoU, "loss":weighted_categorical_crossentropy(np.asarray([0.5,1.,1.5], dtype=np.float32))})
wm_model = tf.keras.models.load_model(path_model2, custom_objects={"meanIoU": meanIoU, "cropIoU": cropIoU, "loss":weighted_categorical_crossentropy(np.asarray([0.5,1.,2], dtype=np.float32))})
models_list.append(wh_model)
models_list.append(wm_model)


The following cell builds the __models_images_filenames__ list containing all the lists of all filenames needed for prediction.

In [ ]:
bool_arr_test = []
bool_arr_test.append([True, "Bipbip", "Haricot", ".jpg"])
bool_arr_test.append([True, "Bipbip", "Mais", ".jpg"])
bool_arr_test.append([True, "Pead", "Haricot", ".jpg"])
bool_arr_test.append([True, "Pead", "Mais", ".jpg"])
bool_arr_test.append([True, "Roseau", "Haricot", ".png"])
bool_arr_test.append([True, "Roseau", "Mais", ".png"])
bool_arr_test.append([True, "Weedelec", "Haricot", ".jpg"])
bool_arr_test.append([True, "Weedelec", "Mais", ".jpg"])

for i in range(0,8):
  team_images = []
  if bool_arr_test[i][0]:
    if i == 1 or i == 4 or i == 5 or i == 7:
      base_folder = os.path.join(cwd, "Test_Final")
    else:
      base_folder = os.path.join(cwd, dataset_version, "Test_Final")
    team = []
    crop = []
    names = []
    base_curr = os.path.join(base_folder, bool_arr_test[i][1], bool_arr_test[i][2])
    fn_images = [x for x in os.listdir(os.path.join(base_curr, "Images"))]
    fn_images.sort()
    for entry in fn_images:
      names.append(entry[:-4])
    #print(names)

    for j in range(0, len(fn_images)):
        team.append(bool_arr_test[i][1])
        crop.append(bool_arr_test[i][2])
    for index, value in enumerate(fn_images):
      fn_images[index] = os.path.join(base_curr, "Images", value)

    zipped_list = list(zip(fn_images, team, crop, names))

    team_images += zipped_list
    models_images_filenames.append(team_images)


## Start Testing
Once both the lists are ready we can start making predictions on the test images. This cell is specifically implemented for the kind of models that we are using, in case different models are loaded think of readapting it. This is due to the fact that some models need different preprocessing on the test images that they are predictinf but also different images since they may be tiled or not.

In [ ]:
# json generation
# ---------------

# this directory is where the submission.json file containing the results will be stored
exp_dir = '/content/drive/My Drive/challenge2/2_phase/Results'

# size of the images used at training time for images that are not cropped
img_w = 1344
img_h = 960

submission_dict = {}

for i in range(0, 8):
  print(str(i))
  model = models_list[i]
  test_images = models_images_filenames[i]
  
  if i == 1 or i == 4 or i == 5 or i == 7: # models that need full images
    for entry in test_images:
      image = Image.open(entry[0])
      width, height = image.size

      # resize image and create crops
      image = image.resize((img_w, img_h))
      img_arr = np.array(image)

      out_sigmoid = model.predict(x=tf.keras.applications.vgg16.preprocess_input(tf.expand_dims(img_arr, 0)))
      resized_sigmoid = tf.image.resize(out_sigmoid, [height,width], method='nearest')

      predicted_class = tf.argmax(resized_sigmoid, -1)
      predicted_class = predicted_class[0, ...]

      img_name = entry[3]

      mask_arr = np.array(predicted_class)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = entry[1]
      submission_dict[img_name]['crop'] = entry[2]
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed
  else: # models that need cropped images
    for entry in test_images:
      image = Image.open(entry[0])
      width, height = image.size

      # resize image and create crops
      image = image.resize(((width // tile_size)*tile_size, (height // tile_size)*tile_size))
      img_arr = np.array(image)
      image_crops = get_patches(img_arr, size=tile_size, stride=tile_size)

      # prediction on each tile stacking each result
      tile_mask_list = []
      for j in range(len(image_crops)):
        tile_arr = image_crops[j]
        
        tile_arr = tile_arr * 1. / 255
        out_sigmoid = model.predict(x=tf.expand_dims(tile_arr, 0))
        
        predicted_class = tf.argmax(out_sigmoid, -1)
        predicted_class = predicted_class[0, ...]

        tile_mask_list.append(np.array(tf.expand_dims(predicted_class, axis=-1)))
      
      mask_crops = np.stack(tile_mask_list)

      # reconstruct and resize
      mask_reconstructed = reconstruct_from_patches(mask_crops, org_img_size=(image.height, image.width), stride=tile_size, size=tile_size)
      
      disegno = np.zeros((image.height, image.width , 3))
      disegno[np.where(mask_reconstructed[0,...,0] == 1)] = [255, 255, 255]
      disegno[np.where(mask_reconstructed[0,...,0] == 0)] = [0,0,0]
      disegno[np.where(mask_reconstructed[0,...,0] == 2)] = [216, 67, 82]

      imm = Image.fromarray(np.uint8(disegno)).resize((width, height))
      mask_arr = np.array(imm)

      new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

      new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
      new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2

      img_name = entry[3]

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = new_mask_arr.shape
      submission_dict[img_name]['team'] = entry[1]
      submission_dict[img_name]['crop'] = entry[2]
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(new_mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(new_mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

# Finally, save the results into the submission.json file
with open(os.path.join(exp_dir, "submission.json"), 'w') as f:
  json.dump(submission_dict, f)